# Doc2Vec
- 작성 일시: 2018-06-08
- 수정 일시: 2018-06-08
- 작성자: 부현경 (hyunkyung.boo@gmail.com)

## 3. Doc2Vec 모델 생성
- 생성된 코퍼스를 불러와 Doc2Vec 모델을 생성한다.

#### 3.1 DB에서 데이터(코퍼스 결과) 출력하기

In [1]:
import mysql.connector
import pandas as pd

In [2]:
table_config = {
    'user': 'root',
    'password': '1234',
    'host': 'localhost',
    'port': 3306,
    'database': 'db_test',
    'raise_on_warnings': True,
    'charset' : 'utf8'
}

try:
    conn = mysql.connector.connect(**table_config)
    curs = conn.cursor()

# 백업용
#     save_path = "D:\\Train_defined_naver_movie_information_add_corpus_result.xlsx"
#     writer = pd.ExcelWriter(save_path)
#     df.to_excel(writer, 'Sheet1', header=True, index=False)
#     writer.save()
    
    # data_type이 'Train'이고 tokenized_user_review의 값이 공백이 아닌 경우만 추출한다.
    sql_select_data = "select idx, tokenized_user_review from naver_movie_info where data_type = 'Train' and tokenized_user_review != \"\"" 
    df = pd.read_sql(sql_select_data, con=conn, columns=True)

except Exception as e:
    print(e)
    
finally:
    conn.close()

In [3]:
# 영어, 숫자는 형태소 분석에서 제외되어 형태소 분석 후 공백이 있을 수 있다.
# 따라서 모델 학습에 사용되는 데이터는 tokenized_user_review가 공백이 아닐 때만 사용한다.
print(df.head(1))

# 총 81,343건.
# print(df.shape)

   idx tokenized_user_review
0    1         번, 보다, 볼때, 재다


In [4]:
# import nltk
# from collections import Counter

# words_list = []
# review_word = []
# tmp = ""
# for r in df['tokenized_user_review']:
#     review_word.append(" ".join(r))
    
# for r in review_word:
#     words_list.append(" ".join(r))
    
# counts = Counter(words_list)
# print(counts,)

#### 3.2 Doc2Vec 모델 생성하기

In [5]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
import multiprocessing
import logging
import time

# Doc2Vec 모델시 사용되는 함수들이다.

# 문서와 문서 ID를 태깅
def TaggedDocumentation(corpus_df, tag_df):
    custom_taggedDocument = []
    corpus_list = []
    for corpus, tag in zip(corpus_df, tag_df):
        splited_corpus = corpus.split(", ")
        custom_taggedDocument.append(TaggedDocument(list(splited_corpus), tags=[tag]))
        # print(TaggedDocumentation)
    # print(TaggedDocumentation[3:7])
    return custom_taggedDocument


# 모델을 생성한다. config를 이용해 모델 세팅을 할 수 있음
def create_doc2vec_model(TaggedDocumentation, config):
    model = Doc2Vec(**config)
    model.build_vocab(TaggedDocumentation)

    # epoch 설정 후 학습
    start = time.time()
    for epoch in range(model.epochs):
        model.train(TaggedDocumentation, total_examples=len(TaggedDocumentation), epochs=model.epochs)
        model.alpha -= 0.002  # decrease the learning rate
        model.min_alpha = model.alpha  # fix the learning rate, no decay

    end = time.time()
    print("During Time: {}".format(end - start))
    return model


# 생성된 모델 저장
# ex. save_path = 'model/Doc2vec(dbow+w,d300,n10,hs,w8,mc20,s0.001,t24).model'
save_root = 'D:\\Doc2Vec_model_20180608\\'
def save_doc2vec_model(model, save_path):
    model.save(save_path)
    print(save_path + "에 저장 완료")

C:\Users\User\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [6]:
#태그는 idx를 이용하며 TaggedDocumentation()를 이용해 문서와 태그를 매핑시켜준다.
TaggedDocuments = TaggedDocumentation(df['tokenized_user_review'], df['idx'])
print(TaggedDocuments[10:11])
# print(len(TaggedDocuments))

[TaggedDocument(words=['눈물', '난', '할머니', '힘내다', '감동', '케미', '좋다'], tags=[11])]


#### Doc2Vec 모델 생성시
- for을 이용하여 config만 바꾸어 설정할 수 있다.
- '(번외) 모델 생성시 config 셋팅' 참고

In [8]:
# ** 300차원, 단어 앞뒤로 5번 읽기, 학습률 0.25, epoch 10번 등, 'sample': 1e-4추가
# 1) 모델 생성
# CPU 코어 사용과 학습시 반환할 로깅을 설정
cores = multiprocessing.cpu_count()
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

# Doc2Vec 파라매터 설정
config = {
    'dm': 0,  # PV-DBOW / default 1
    'dbow_words': 1,  # w2v simultaneous with DBOW d2v / default 0
    'window': 5,  # distance between the predicted word and context words
    'vector_size': 300,  # vector size
    'alpha': 0.025,  # learning-rate
    'sample': 1e-4,
    'seed': 1234,
    'min_count': 20,  # ignore with freq lower
    'min_alpha': 0.025,  # min learning-rate
    'workers': cores,  # multi cpu
    'hs': 1,  # hierarchical softmax / default 0
    'negative': 20,  # negative sampling / default 5
    'epochs': 10,  # 보통 딥러닝에서 말하는 epoch과 비슷한, 반복 횟수. 또한 모델 학습시 반복 횟수 만큼 반복된다!!!
}

kyobo_model = create_doc2vec_model(TaggedDocuments, config)

# 2) 모델 저장
doc2vec_save_path = save_root+'0407_Doc2vec(dbow+w,d300,n20,hs,w5,mc20,s0.001,e10).model'
save_doc2vec_model(kyobo_model, doc2vec_save_path)

2018-06-08 17:44:25,848 : INFO : collecting all words and their counts
2018-06-08 17:44:25,851 : INFO : PROGRESS: at example #0, processed 0 words (0/s), 0 word types, 0 tags
2018-06-08 17:44:25,910 : INFO : PROGRESS: at example #10000, processed 117703 words (2160910/s), 12350 word types, 10048 tags
2018-06-08 17:44:25,950 : INFO : PROGRESS: at example #20000, processed 237491 words (2993363/s), 17288 word types, 20088 tags
2018-06-08 17:44:25,990 : INFO : PROGRESS: at example #30000, processed 357715 words (3122096/s), 20929 word types, 30128 tags
2018-06-08 17:44:26,030 : INFO : PROGRESS: at example #40000, processed 476622 words (2975405/s), 23854 word types, 40168 tags
2018-06-08 17:44:26,063 : INFO : PROGRESS: at example #50000, processed 594769 words (3722464/s), 26250 word types, 50216 tags
2018-06-08 17:44:26,103 : INFO : PROGRESS: at example #60000, processed 714605 words (3049773/s), 28391 word types, 60252 tags
2018-06-08 17:44:26,147 : INFO : PROGRESS: at example #70000, p

2018-06-08 17:45:06,225 : INFO : worker thread finished; awaiting finish of 1 more threads
2018-06-08 17:45:06,228 : INFO : worker thread finished; awaiting finish of 0 more threads
2018-06-08 17:45:06,229 : INFO : EPOCH - 5 : training on 967424 raw words (530549 effective words) took 7.2s, 73242 effective words/s
2018-06-08 17:45:07,379 : INFO : EPOCH 6 - PROGRESS: at 13.53% examples, 63055 words/s, in_qsize 7, out_qsize 0
2018-06-08 17:45:08,529 : INFO : EPOCH 6 - PROGRESS: at 29.90% examples, 69586 words/s, in_qsize 7, out_qsize 0
2018-06-08 17:45:09,705 : INFO : EPOCH 6 - PROGRESS: at 46.34% examples, 71230 words/s, in_qsize 7, out_qsize 0
2018-06-08 17:45:10,877 : INFO : EPOCH 6 - PROGRESS: at 62.98% examples, 72152 words/s, in_qsize 7, out_qsize 0
2018-06-08 17:45:11,988 : INFO : EPOCH 6 - PROGRESS: at 79.51% examples, 73488 words/s, in_qsize 7, out_qsize 0
2018-06-08 17:45:13,161 : INFO : EPOCH 6 - PROGRESS: at 96.05% examples, 73638 words/s, in_qsize 4, out_qsize 0
2018-06-08 1

2018-06-08 17:45:56,199 : INFO : EPOCH 2 - PROGRESS: at 92.92% examples, 64023 words/s, in_qsize 7, out_qsize 0
2018-06-08 17:45:56,495 : INFO : worker thread finished; awaiting finish of 3 more threads
2018-06-08 17:45:56,602 : INFO : worker thread finished; awaiting finish of 2 more threads
2018-06-08 17:45:56,611 : INFO : worker thread finished; awaiting finish of 1 more threads
2018-06-08 17:45:56,622 : INFO : worker thread finished; awaiting finish of 0 more threads
2018-06-08 17:45:56,623 : INFO : EPOCH - 2 : training on 967424 raw words (530739 effective words) took 8.1s, 65270 effective words/s
2018-06-08 17:45:57,651 : INFO : EPOCH 3 - PROGRESS: at 11.54% examples, 59614 words/s, in_qsize 7, out_qsize 0
2018-06-08 17:45:58,755 : INFO : EPOCH 3 - PROGRESS: at 23.79% examples, 59625 words/s, in_qsize 7, out_qsize 0
2018-06-08 17:45:59,818 : INFO : EPOCH 3 - PROGRESS: at 38.13% examples, 63711 words/s, in_qsize 8, out_qsize 0
2018-06-08 17:46:00,851 : INFO : EPOCH 3 - PROGRESS: a

2018-06-08 17:46:48,197 : INFO : EPOCH - 8 : training on 967424 raw words (529808 effective words) took 7.7s, 68900 effective words/s
2018-06-08 17:46:49,414 : INFO : EPOCH 9 - PROGRESS: at 13.53% examples, 59411 words/s, in_qsize 8, out_qsize 0
2018-06-08 17:46:50,669 : INFO : EPOCH 9 - PROGRESS: at 24.81% examples, 53498 words/s, in_qsize 7, out_qsize 0
2018-06-08 17:46:51,674 : INFO : EPOCH 9 - PROGRESS: at 32.96% examples, 50590 words/s, in_qsize 8, out_qsize 0
2018-06-08 17:46:52,842 : INFO : EPOCH 9 - PROGRESS: at 43.22% examples, 49686 words/s, in_qsize 7, out_qsize 0
2018-06-08 17:46:53,862 : INFO : EPOCH 9 - PROGRESS: at 52.63% examples, 49416 words/s, in_qsize 8, out_qsize 0
2018-06-08 17:46:54,872 : INFO : EPOCH 9 - PROGRESS: at 66.09% examples, 52576 words/s, in_qsize 8, out_qsize 0
2018-06-08 17:46:55,881 : INFO : EPOCH 9 - PROGRESS: at 77.45% examples, 53544 words/s, in_qsize 7, out_qsize 0
2018-06-08 17:46:56,943 : INFO : EPOCH 9 - PROGRESS: at 89.83% examples, 54539 wor

2018-06-08 17:47:38,898 : INFO : EPOCH - 4 : training on 967424 raw words (530704 effective words) took 8.7s, 61268 effective words/s
2018-06-08 17:47:40,031 : INFO : EPOCH 5 - PROGRESS: at 9.45% examples, 44346 words/s, in_qsize 7, out_qsize 0
2018-06-08 17:47:41,065 : INFO : EPOCH 5 - PROGRESS: at 21.73% examples, 53554 words/s, in_qsize 7, out_qsize 0
2018-06-08 17:47:42,066 : INFO : EPOCH 5 - PROGRESS: at 32.96% examples, 55697 words/s, in_qsize 8, out_qsize 0
2018-06-08 17:47:43,133 : INFO : EPOCH 5 - PROGRESS: at 41.18% examples, 52012 words/s, in_qsize 7, out_qsize 0
2018-06-08 17:47:44,225 : INFO : EPOCH 5 - PROGRESS: at 54.74% examples, 54728 words/s, in_qsize 7, out_qsize 0
2018-06-08 17:47:45,243 : INFO : EPOCH 5 - PROGRESS: at 65.03% examples, 54526 words/s, in_qsize 7, out_qsize 0
2018-06-08 17:47:46,258 : INFO : EPOCH 5 - PROGRESS: at 74.25% examples, 53733 words/s, in_qsize 7, out_qsize 0
2018-06-08 17:47:47,260 : INFO : EPOCH 5 - PROGRESS: at 85.74% examples, 54496 word

...

2018-06-08 17:55:54,954 : INFO : EPOCH 1 - PROGRESS: at 29.90% examples, 74729 words/s, in_qsize 7, out_qsize 0
2018-06-08 17:55:55,974 : INFO : EPOCH 1 - PROGRESS: at 45.29% examples, 76671 words/s, in_qsize 7, out_qsize 0
2018-06-08 17:55:56,976 : INFO : EPOCH 1 - PROGRESS: at 58.85% examples, 75328 words/s, in_qsize 8, out_qsize 0
2018-06-08 17:55:58,024 : INFO : EPOCH 1 - PROGRESS: at 74.25% examples, 75953 words/s, in_qsize 7, out_qsize 0
2018-06-08 17:55:59,107 : INFO : EPOCH 1 - PROGRESS: at 90.93% examples, 76814 words/s, in_qsize 7, out_qsize 0
2018-06-08 17:55:59,455 : INFO : worker thread finished; awaiting finish of 3 more threads
2018-06-08 17:55:59,512 : INFO : worker thread finished; awaiting finish of 2 more threads
2018-06-08 17:55:59,598 : INFO : worker thread finished; awaiting finish of 1 more threads
2018-06-08 17:55:59,612 : INFO : worker thread finished; awaiting finish of 0 more threads
2018-06-08 17:55:59,613 : INFO : EPOCH - 1 : training on 967424 raw words (5

2018-06-08 17:56:43,923 : INFO : EPOCH 8 - PROGRESS: at 13.53% examples, 66883 words/s, in_qsize 7, out_qsize 0
2018-06-08 17:56:45,026 : INFO : EPOCH 8 - PROGRESS: at 29.90% examples, 73135 words/s, in_qsize 8, out_qsize 0
2018-06-08 17:56:46,072 : INFO : EPOCH 8 - PROGRESS: at 42.21% examples, 69836 words/s, in_qsize 7, out_qsize 0
2018-06-08 17:56:47,096 : INFO : EPOCH 8 - PROGRESS: at 57.78% examples, 72379 words/s, in_qsize 8, out_qsize 0
2018-06-08 17:56:48,205 : INFO : EPOCH 8 - PROGRESS: at 74.25% examples, 73725 words/s, in_qsize 7, out_qsize 0
2018-06-08 17:56:49,308 : INFO : EPOCH 8 - PROGRESS: at 90.93% examples, 74691 words/s, in_qsize 7, out_qsize 0
2018-06-08 17:56:49,597 : INFO : worker thread finished; awaiting finish of 3 more threads
2018-06-08 17:56:49,688 : INFO : worker thread finished; awaiting finish of 2 more threads
2018-06-08 17:56:49,742 : INFO : worker thread finished; awaiting finish of 1 more threads
2018-06-08 17:56:49,789 : INFO : worker thread finished

During Time: 760.9719998836517


2018-06-08 17:57:09,209 : INFO : saved D:\Doc2Vec_model_20180608\0407_Doc2vec(dbow+w,d300,n20,hs,w5,mc20,s0.001,e10).model


D:\Doc2Vec_model_20180608\0407_Doc2vec(dbow+w,d300,n20,hs,w5,mc20,s0.001,e10).model에 저장 완료


In [9]:
# ** 300차원, 단어 앞뒤로 5번 읽기, 학습률 0.25, epoch 10번 등, 'sample': 1e-4추가
# 1) 모델 생성
# CPU 코어 사용과 학습시 반환할 로깅을 설정
cores = multiprocessing.cpu_count()
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

# Doc2Vec 파라매터 설정
config = {
    'dm': 0,  # PV-DBOW / default 1
    'dbow_words': 1,  # w2v simultaneous with DBOW d2v / default 0
    'window': 5,  # distance between the predicted word and context words
    'vector_size': 300,  # vector size
    'alpha': 0.025,  # learning-rate
    'sample': 1e-2,
    'seed': 1234,
    'min_count': 20,  # ignore with freq lower
    'min_alpha': 0.025,  # min learning-rate
    'workers': cores,  # multi cpu
    'hs': 1,  # hierarchical softmax / default 0
    'negative': 20,  # negative sampling / default 5
    'epochs': 10,  # 보통 딥러닝에서 말하는 epoch과 비슷한, 반복 횟수. 또한 모델 학습시 반복 횟수 만큼 반복된다!!!
}

kyobo_model = create_doc2vec_model(TaggedDocuments, config)

# 2) 모델 저장
doc2vec_save_path = save_root+'0408_Doc2vec(dbow+w,d300,n20,hs,w5,mc20,s0.001,e10).model'
save_doc2vec_model(kyobo_model, doc2vec_save_path)

2018-06-08 18:01:46,219 : INFO : collecting all words and their counts
2018-06-08 18:01:46,224 : INFO : PROGRESS: at example #0, processed 0 words (0/s), 0 word types, 0 tags
2018-06-08 18:01:46,276 : INFO : PROGRESS: at example #10000, processed 117703 words (2325212/s), 12350 word types, 10048 tags
2018-06-08 18:01:46,319 : INFO : PROGRESS: at example #20000, processed 237491 words (2996329/s), 17288 word types, 20088 tags
2018-06-08 18:01:46,350 : INFO : PROGRESS: at example #30000, processed 357715 words (3991805/s), 20929 word types, 30128 tags
2018-06-08 18:01:46,390 : INFO : PROGRESS: at example #40000, processed 476622 words (3066188/s), 23854 word types, 40168 tags
2018-06-08 18:01:46,425 : INFO : PROGRESS: at example #50000, processed 594769 words (3403819/s), 26250 word types, 50216 tags
2018-06-08 18:01:46,462 : INFO : PROGRESS: at example #60000, processed 714605 words (3320625/s), 28391 word types, 60252 tags
2018-06-08 18:01:46,500 : INFO : PROGRESS: at example #70000, p

2018-06-08 18:02:40,258 : INFO : worker thread finished; awaiting finish of 2 more threads
2018-06-08 18:02:40,327 : INFO : worker thread finished; awaiting finish of 1 more threads
2018-06-08 18:02:40,356 : INFO : worker thread finished; awaiting finish of 0 more threads
2018-06-08 18:02:40,357 : INFO : EPOCH - 3 : training on 967424 raw words (941353 effective words) took 17.7s, 53044 effective words/s
2018-06-08 18:02:41,836 : INFO : EPOCH 4 - PROGRESS: at 5.24% examples, 33014 words/s, in_qsize 8, out_qsize 0
2018-06-08 18:02:42,919 : INFO : EPOCH 4 - PROGRESS: at 11.54% examples, 41869 words/s, in_qsize 7, out_qsize 0
2018-06-08 18:02:44,149 : INFO : EPOCH 4 - PROGRESS: at 17.64% examples, 43587 words/s, in_qsize 8, out_qsize 0
2018-06-08 18:02:45,273 : INFO : EPOCH 4 - PROGRESS: at 24.81% examples, 47483 words/s, in_qsize 8, out_qsize 0
2018-06-08 18:02:46,521 : INFO : EPOCH 4 - PROGRESS: at 31.96% examples, 48887 words/s, in_qsize 8, out_qsize 0
2018-06-08 18:02:47,820 : INFO : 

2018-06-08 18:03:46,669 : INFO : EPOCH 8 - PROGRESS: at 5.32% examples, 36624 words/s, in_qsize 8, out_qsize 0
2018-06-08 18:03:47,711 : INFO : EPOCH 8 - PROGRESS: at 12.52% examples, 49165 words/s, in_qsize 7, out_qsize 0
2018-06-08 18:03:48,761 : INFO : EPOCH 8 - PROGRESS: at 19.69% examples, 53903 words/s, in_qsize 7, out_qsize 0
2018-06-08 18:03:49,797 : INFO : EPOCH 8 - PROGRESS: at 26.82% examples, 56618 words/s, in_qsize 8, out_qsize 0
2018-06-08 18:03:50,889 : INFO : EPOCH 8 - PROGRESS: at 33.98% examples, 57739 words/s, in_qsize 7, out_qsize 0
2018-06-08 18:03:52,110 : INFO : EPOCH 8 - PROGRESS: at 42.21% examples, 58813 words/s, in_qsize 8, out_qsize 0
2018-06-08 18:03:53,219 : INFO : EPOCH 8 - PROGRESS: at 50.51% examples, 60399 words/s, in_qsize 8, out_qsize 0
2018-06-08 18:03:54,313 : INFO : EPOCH 8 - PROGRESS: at 58.85% examples, 61709 words/s, in_qsize 7, out_qsize 0
2018-06-08 18:03:55,395 : INFO : EPOCH 8 - PROGRESS: at 67.10% examples, 62815 words/s, in_qsize 7, out_q

2018-06-08 18:04:47,060 : INFO : EPOCH 2 - PROGRESS: at 38.13% examples, 64457 words/s, in_qsize 7, out_qsize 0
2018-06-08 18:04:48,125 : INFO : EPOCH 2 - PROGRESS: at 46.34% examples, 65825 words/s, in_qsize 7, out_qsize 0
2018-06-08 18:04:49,201 : INFO : EPOCH 2 - PROGRESS: at 54.74% examples, 66732 words/s, in_qsize 8, out_qsize 0
2018-06-08 18:04:50,400 : INFO : EPOCH 2 - PROGRESS: at 62.98% examples, 66489 words/s, in_qsize 7, out_qsize 0
2018-06-08 18:04:51,467 : INFO : EPOCH 2 - PROGRESS: at 71.17% examples, 67182 words/s, in_qsize 7, out_qsize 0
2018-06-08 18:04:52,477 : INFO : EPOCH 2 - PROGRESS: at 78.50% examples, 67223 words/s, in_qsize 8, out_qsize 0
2018-06-08 18:04:53,593 : INFO : EPOCH 2 - PROGRESS: at 86.78% examples, 67475 words/s, in_qsize 7, out_qsize 0
2018-06-08 18:04:54,675 : INFO : EPOCH 2 - PROGRESS: at 95.01% examples, 67848 words/s, in_qsize 5, out_qsize 0
2018-06-08 18:04:54,915 : INFO : worker thread finished; awaiting finish of 3 more threads
2018-06-08 18

2018-06-08 18:05:50,185 : INFO : worker thread finished; awaiting finish of 3 more threads
2018-06-08 18:05:50,187 : INFO : worker thread finished; awaiting finish of 2 more threads
2018-06-08 18:05:50,332 : INFO : worker thread finished; awaiting finish of 1 more threads
2018-06-08 18:05:50,450 : INFO : worker thread finished; awaiting finish of 0 more threads
2018-06-08 18:05:50,451 : INFO : EPOCH - 6 : training on 967424 raw words (941553 effective words) took 13.7s, 68640 effective words/s
2018-06-08 18:05:51,556 : INFO : EPOCH 7 - PROGRESS: at 5.32% examples, 44419 words/s, in_qsize 7, out_qsize 0
2018-06-08 18:05:52,680 : INFO : EPOCH 7 - PROGRESS: at 13.53% examples, 56846 words/s, in_qsize 8, out_qsize 0
2018-06-08 18:05:53,767 : INFO : EPOCH 7 - PROGRESS: at 21.73% examples, 61592 words/s, in_qsize 7, out_qsize 0
2018-06-08 18:05:54,923 : INFO : EPOCH 7 - PROGRESS: at 29.90% examples, 63020 words/s, in_qsize 7, out_qsize 0
2018-06-08 18:05:56,015 : INFO : EPOCH 7 - PROGRESS: a

...

2018-06-08 18:24:00,473 : INFO : worker thread finished; awaiting finish of 0 more threads
2018-06-08 18:24:00,474 : INFO : EPOCH - 4 : training on 967424 raw words (941552 effective words) took 16.7s, 56462 effective words/s
2018-06-08 18:24:01,630 : INFO : EPOCH 5 - PROGRESS: at 5.32% examples, 42384 words/s, in_qsize 7, out_qsize 0
2018-06-08 18:24:03,071 : INFO : EPOCH 5 - PROGRESS: at 13.53% examples, 48740 words/s, in_qsize 7, out_qsize 0
2018-06-08 18:24:04,256 : INFO : EPOCH 5 - PROGRESS: at 21.73% examples, 53995 words/s, in_qsize 7, out_qsize 0
2018-06-08 18:24:05,494 : INFO : EPOCH 5 - PROGRESS: at 29.90% examples, 56153 words/s, in_qsize 8, out_qsize 0
2018-06-08 18:24:06,868 : INFO : EPOCH 5 - PROGRESS: at 37.08% examples, 54723 words/s, in_qsize 8, out_qsize 0
2018-06-08 18:24:08,012 : INFO : EPOCH 5 - PROGRESS: at 45.29% examples, 56745 words/s, in_qsize 8, out_qsize 0
2018-06-08 18:24:09,031 : INFO : EPOCH 5 - PROGRESS: at 52.63% examples, 57928 words/s, in_qsize 7, out

2018-06-08 18:25:09,167 : INFO : EPOCH 8 - PROGRESS: at 100.00% examples, 43832 words/s, in_qsize 0, out_qsize 1
2018-06-08 18:25:09,168 : INFO : worker thread finished; awaiting finish of 0 more threads
2018-06-08 18:25:09,170 : INFO : EPOCH - 8 : training on 967424 raw words (941416 effective words) took 21.5s, 43825 effective words/s
2018-06-08 18:25:10,645 : INFO : EPOCH 9 - PROGRESS: at 5.32% examples, 33262 words/s, in_qsize 7, out_qsize 0
2018-06-08 18:25:12,181 : INFO : EPOCH 9 - PROGRESS: at 12.52% examples, 38849 words/s, in_qsize 8, out_qsize 0
2018-06-08 18:25:13,448 : INFO : EPOCH 9 - PROGRESS: at 18.67% examples, 40935 words/s, in_qsize 8, out_qsize 0
2018-06-08 18:25:14,489 : INFO : EPOCH 9 - PROGRESS: at 23.79% examples, 42061 words/s, in_qsize 8, out_qsize 0
2018-06-08 18:25:15,709 : INFO : EPOCH 9 - PROGRESS: at 28.86% examples, 41616 words/s, in_qsize 7, out_qsize 0
2018-06-08 18:25:16,876 : INFO : EPOCH 9 - PROGRESS: at 32.96% examples, 40355 words/s, in_qsize 8, ou

During Time: 1445.2309997081757


2018-06-08 18:25:53,763 : INFO : saved D:\Doc2Vec_model_20180608\0408_Doc2vec(dbow+w,d300,n20,hs,w5,mc20,s0.001,e10).model


D:\Doc2Vec_model_20180608\0408_Doc2vec(dbow+w,d300,n20,hs,w5,mc20,s0.001,e10).model에 저장 완료


In [11]:
print('finished!')

finished!
